In [1]:
import json
import time
from tqdm import tqdm
import torch
from torch.profiler import profile, record_function, ProfilerActivity
import pandas as pd
from datasets import load_dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from torchao.quantization import (
    quantize_,
    Int8DynamicActivationInt8WeightConfig,
    Float8DynamicActivationFloat8WeightConfig,
    Float8DynamicActivationInt4WeightConfig,
    Int8DynamicActivationInt4WeightConfig,
)
import jiwer
from jiwer import (
    Compose,
    ToLowerCase,
    RemoveMultipleSpaces,
    Strip,
)
from IPython.display import Audio

/home/d_alshevskiy/whisper/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def asr_metrics(hypothesis: str, reference: str):
    tr = Compose([ToLowerCase(), RemoveMultipleSpaces(), Strip()])

    ref_tr = tr(reference)
    hyp_tr = tr(hypothesis)

    out = jiwer.process_words(ref_tr, hyp_tr)
    wer = out.wer
    # S, D, I = out.substitutions, out.deletions, out.insertions

    cer = jiwer.cer(ref_tr, hyp_tr) # ?????

    return {
        "wer": wer,
        "cer": cer,
    }

In [3]:
dataset = load_dataset("bond005/sberdevices_golos_10h_crowd", split="validation") #, split="test")

In [4]:
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v3")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3")
forced_decoder_ids = processor.get_decoder_prompt_ids(language="russian", task="transcribe")
model.eval()

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1280)
      (layers): ModuleList(
        (0-31): 32 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear(in_features=5120, out_features=1280, bias=Tr

In [5]:
quantize_(model, Int8DynamicActivationInt4WeightConfig())

/home/d_alshevskiy/whisper/lib/python3.11/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)


In [6]:
model.eval()

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1280)
      (layers): ModuleList(
        (0-31): 32 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, weight=LinearActivationQuantizedTensor(activation=<function _int8_asymm_per_token_quant at 0x7a99aa642b60>, weight=AffineQuantizedTensor(shape=torch.Size([1280, 1280]), block_size=(1, 32), device=cpu, _layout=PlainLayout(), tensor_impl_dtype=torch.int8, quant_min=-8, quant_max=7)))
            (v_proj): Linear(in_features=1280, out_features=1280, weight=LinearActivationQuantizedTensor(activation=<function _int8_asymm_per_token_quant at 0x7a99aa642b60>, weight=AffineQuantizedTensor(shape=torch.Size([1280, 1280]), block_size=(

In [7]:
gc = model.generation_config
gc.language = "ru"
gc.task = "transcribe"

In [8]:
results = []
i = 0
for audio in tqdm(dataset):
    audio_array = audio["audio"]["array"]
    sampling_rate = audio["audio"]["sampling_rate"]
    reference = audio["transcription"]

    start_time = time.time()
    input_features = processor(audio_array, sampling_rate=sampling_rate, return_tensors="pt").input_features 
    predicted_ids = model.generate(input_features, generation_config=gc)[0] #Уточнить в зависимости от выбранной модели
    hypothesis = processor.decode(predicted_ids)
    run_time = time.time() - start_time
    metrics = asr_metrics(hypothesis, reference)
    metrics["run_time_sec"] = run_time
    if i % 50 == 0:
        print("referenct:")
        print(reference)
        print("hypothesis:")
        print(hypothesis)
    i += 1
    results.append(metrics)

  0%|          | 1/793 [00:20<4:36:55, 20.98s/it]

referenct:
можешь включить сериал теория большого взрыва
hypothesis:
 Можешь включить сериал «Теория большого взрыва»?


  6%|▋         | 51/793 [08:47<2:12:40, 10.73s/it]

referenct:
покажи на смотрешке канал бридж тв
hypothesis:
 Покажи на Смотрёжке канал Бридж ТВ.


 13%|█▎        | 101/793 [17:13<1:42:23,  8.88s/it]

...

 88%|████████▊ | 701/793 [2:03:44<15:11,  9.91s/it]

referenct:
вячеслав владимирович месяцев
hypothesis:
 Вячеслав Владимирович Месяцев


 95%|█████████▍| 751/793 [2:12:23<07:10, 10.25s/it]

referenct:
футбольный матч тоттенхэм лестер
hypothesis:
 Футбольный матч Тоттенхэм Лестер


100%|██████████| 793/793 [2:19:52<00:00, 10.58s/it]


In [ ]:
# results = []
# i = 0
# for audio in tqdm(dataset):
#     audio_array = audio["audio"]["array"]
#     sampling_rate = audio["audio"]["sampling_rate"]
#     reference = audio["transcription"]

#     start_time = time.time()
#     input_features = processor(audio_array, sampling_rate=sampling_rate, return_tensors="pt").input_features 
#     predicted_ids = model.generate(input_features, generation_config=gc)[0] #Уточнить в зависимости от выбранной модели
#     hypothesis = processor.decode(predicted_ids)
#     run_time = time.time() - start_time
#     metrics = asr_metrics(hypothesis, reference)
#     metrics["run_time_sec"] = run_time
#     # if i % 50 == 0:
#     #     print("referenct:")
#     #     print(reference)
#     #     print("hypothesis:")
#     #     print(hypothesis)
#     # i += 1
#     results.append(metrics)

  0%|          | 0/793 [00:00<?, ?it/s]

  0%|          | 1/793 [00:06<1:28:48,  6.73s/it]

referenct:
можешь включить сериал теория большого взрыва
hypothesis:
 Можешь включить сериал «Теория большого взрыва»?


  6%|▋         | 51/793 [04:06<59:20,  4.80s/it]  

referenct:
покажи на смотрешке канал бридж тв
hypothesis:
 Покажи на Смотрёжке канал Бридж ТВ.


...

 88%|████████▊ | 701/793 [56:46<06:35,  4.30s/it]

referenct:
вячеслав владимирович месяцев
hypothesis:
 Вячеслав Владимирович Месяцев


 95%|█████████▍| 751/793 [1:00:43<03:11,  4.56s/it]

referenct:
футбольный матч тоттенхэм лестер
hypothesis:
 Футбольный матч Тоттенхэм Лестер


100%|██████████| 793/793 [1:04:13<00:00,  4.86s/it]


In [9]:
df_results = pd.DataFrame(results)

summary = {
    "total_samples": len(df_results),
    "avg_wer": df_results["wer"].mean(),
    "avg_cer": df_results["cer"].mean(),
    "avg_time_per_audio": df_results["run_time_sec"].mean(),
    "total_time": df_results["run_time_sec"].sum(),
}

print("large-v3")
print(json.dumps(summary, ensure_ascii=True, indent=2))

large-v3
{
  "total_samples": 793,
  "avg_wer": 0.44977964518317604,
  "avg_cer": 0.16099429957508785,
  "avg_time_per_audio": 10.5657887552004,
  "total_time": 8378.670482873917
}
